<a href="https://colab.research.google.com/github/BiakkoNoMiko/homework_projects/blob/main/Autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import json

# Text preparation for NLP

In [2]:
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# preproc text to get words only

en_ru_words = re.compile("[a-zа-яё]+")

def words_only(text):
    return " ".join(en_ru_words.findall(text.lower()))

In [11]:
# read file
df = pd.read_excel('/content/Volchkevich_8_klass_Geometria.xlsx')

In [12]:
#preproc text

df['task'] = df['task'].progress_apply(words_only)

my bar!: 100%|██████████| 853/853 [00:00<00:00, 70573.85it/s]


In [13]:
df.head()

,task,answer,solution,paint,topic,subtopic,кол-во звезд
0,на рисунке снизу показана так называемая штурм...,NaN,NaN,2,Параллелограмм,NaN,NaN
1,на рисунке показан шарнирный механизм который ...,NaN,NaN,3,Параллелограмм,NaN,NaN
2,на рисунке справа показаны два колеса паровоза...,NaN,NaN,4,Параллелограмм,NaN,NaN
3,на рисунке слева показано устройство шарнирног...,NaN,NaN,5,Параллелограмм,NaN,NaN
4,нарисуйте две окружности одинакового радиуса с...,NaN,NaN,6,Параллелограмм,Признак,NaN


In [14]:
%%time
vec = TfidfVectorizer(stop_words=stopwords.words('english'),
                     #   max_features=5000,
                        ngram_range=(1,4))
bow = vec.fit_transform(df['task'])

CPU times: user 118 ms, sys: 5.83 ms, total: 124 ms
Wall time: 155 ms


In [15]:
query_list = np.array(vec.get_feature_names_out())
query_list

array(['aa', 'aa bb', 'aa bb cc', ..., 'ящика определите',
       'ящика определите по', 'ящика определите по данным'], dtype=object)

In [46]:
import joblib

filename= 'query_list.sav'
saved_model=joblib.dump(query_dict, filename)

In [16]:
len(query_list)

34266

# Autocomplete

##BERT MASKING NEXT TOKEN

We don't use any large LLM for next token prediction for now, it's because...because they are large. If the project will use casual LLMs for some purposes, we can aplly the same LLM for searching engine.

In [ ]:
!pip install -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9

  Cloning https://github.com/huggingface/transformers (to revision de9255de27abfcae4a1f816b904915f0b1e23cd9) to /tmp/pip-req-build-ercx27jl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ercx27jl
  Running command git rev-parse -q --verify 'sha^de9255de27abfcae4a1f816b904915f0b1e23cd9'
  Running command git fetch -q https://github.com/huggingface/transformers de9255de27abfcae4a1f816b904915f0b1e23cd9
  Running command git checkout -q de9255de27abfcae4a1f816b904915f0b1e23cd9
  Resolved https://github.com/huggingface/transformers to commit de9255de27abfcae4a1f816b904915f0b1e23cd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/

In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

For getting embeddings and semantic search we apply bert-like model, so we can try to predict next token by masking it.

In [ ]:
# Define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
classifier = pipeline("fill-mask", model = "DeepPavlov/rubert-base-cased-sentence", tokenizer=tokenizer)
classifier("двухатомный идеальный [MASK]")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'score': 0.0002246604854008183,
  'token': 85588,
  'token_str': 'пени',
  'sequence': 'двухатомный идеальный пени'},
 {'score': 0.00018175203877035528,
  'token': 38132,
  'token_str': 'советы',
  'sequence': 'двухатомный идеальный советы'},
 {'score': 0.00016879251052159816,
  'token': 3813,
  'token_str': 'дан',
  'sequence': 'двухатомный идеальный дан'},
 {'score': 0.00015720840019639581,
  'token': 3111,
  'token_str': '勤',
  'sequence': 'двухатомный идеальный 勤'},
 {'score': 0.00015274326142389327,
  'token': 18349,
  'token_str': 'призван',
  'sequence': 'двухатомный идеальный призван'}]

FAIL. But fine-tunung may help

##**TRIE**

###Var 1

In [62]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Autocomplete:
    def __init__(self, vocabulary):
        self.root = TrieNode()
        self.build_trie(vocabulary)

    def build_trie(self, vocabulary):
        for word in vocabulary:
            self.insert_word(word)

    def insert_word(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def find_words(self, prefix):
        node = self.root
        result = []
        for char in prefix:
            if char not in node.children:
                return result
            node = node.children[char]
        self.traverse_subtrie(node, prefix, result)
        return result

    def traverse_subtrie(self, node, prefix, result):
        if node.is_end_of_word:
            result.append(prefix)
        for char in node.children:
            self.traverse_subtrie(node.children[char], prefix + char, result)

In [63]:
vocabulary = query_list
autocomplete = Autocomplete(vocabulary)

In [64]:
%%time
prefix = 'площ'
results = autocomplete.find_words(prefix)
print(results[:5])

['площадей', 'площадей двух', 'площадей двух других', 'площадей других', 'площадей жёлтых']
CPU times: user 819 µs, sys: 0 ns, total: 819 µs
Wall time: 814 µs


In [66]:
%%time
prefix = 'кол'
results = autocomplete.find_words(prefix)
print(results[:5])

['колес', 'колес дышло', 'колес дышло будет', 'колес дышло будет всегда', 'колес почему']
CPU times: user 253 µs, sys: 28 µs, total: 281 µs
Wall time: 287 µs


In [72]:
%%time
prefix = 'те'
results = autocomplete.find_words(prefix)
print(results[:5])

['театра', 'театра чтобы', 'театра чтобы зрителям', 'театра чтобы зрителям из', 'тела']
CPU times: user 565 µs, sys: 0 ns, total: 565 µs
Wall time: 571 µs


In [71]:
%%time
prefix = 'кол'
results = autocomplete.find_words(prefix)
print(results[:5])

['колес', 'колес дышло', 'колес дышло будет', 'колес дышло будет всегда', 'колес почему']
CPU times: user 270 µs, sys: 29 µs, total: 299 µs
Wall time: 292 µs




---



###Var 2

In [27]:
class TrieNode:

    def __init__(self, c):
        self.data = c
        self.isEnd = False
        self.children = {} #map

class Trie:
    def __init__(self):
        self.root = TrieNode('')

    #Add a word to trie, Time O(s) Space O(1) s is word length
    def insert(self, word):
        node = self.root
        for char in word:
            if not char in node.children:
                node.children[char] = TrieNode(char)
            node = node.children[char]
        node.isEnd = True

  #find all word with given prefix
	#Time O(n), Space O(n), n is number of nodes involved (include prefix and branches)
    def autocomplete(self, word):
        res = []
        node = self.root
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                return []
        self.helper(node, res, word[:-1]) #except the last "ama", node is "z"
        return res

  #recursive function called by autocomplete
	#Time O(n), Space O(n), n is number of nodes in branches
    def helper(self, node, res, prefix):
        if node.isEnd:
            res.append(prefix + node.data)
        for child in node.children.values():
            self.helper(child, res, prefix + node.data)


In [28]:
t = Trie()
for query in query_list:
  t.insert(query)

In [73]:
%%time
a = t.autocomplete("яр")
print(a[:5])

['яркий', 'яркий солнечный', 'яркий солнечный день', 'яркий солнечный день на']
CPU times: user 174 µs, sys: 0 ns, total: 174 µs
Wall time: 154 µs


In [76]:
%%time
a = t.autocomplete("кол")
print(a[:5])

['колес', 'колес дышло', 'колес дышло будет', 'колес дышло будет всегда', 'колес почему']
CPU times: user 292 µs, sys: 0 ns, total: 292 µs
Wall time: 296 µs


In [31]:
%%time
a = t.autocomplete("колес ды")
print(a[:5])

['колес дышло', 'колес дышло будет', 'колес дышло будет всегда']
CPU times: user 125 µs, sys: 0 ns, total: 125 µs
Wall time: 135 µs


## **Fast_autocomplete**

In [33]:
!pip install fast-autocomplete[levenshtein]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.1 MB/s eta 0:00:00


In [34]:
from fast_autocomplete import AutoComplete

In [36]:
# convert to formart: "{someword:{}, }"
query_dict = {}

for query in query_list:
  query_dict[query] = {}

In [43]:
import joblib

filename= 'query_dict.sav'
saved_model=joblib.dump(query_dict, filename)

In [37]:
words = query_dict
autocomplete = AutoComplete(words=words, valid_chars_for_string = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя') #add characters of rus alphabet

[['колес'], ['колеса'], ['колес при'], ['колес дышло'], ['колес почему']]


In [45]:
%%time
print(autocomplete.search(word='площади', max_cost=5, size = 5))

[['площади'], ['площади вы'], ['площади то'], ['площади всех'], ['площади двух']]
CPU times: user 170 µs, sys: 0 ns, total: 170 µs
Wall time: 154 µs


FAIL.  not good with russian language

## **Using google source for auto-completion feature**

In [50]:
pip install requests_html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.5 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=e6f975b7de32670c62fa8516720637229596d85e40c5e71fd60a8223d1a087d9
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [51]:
import requests
import urllib
import json
import operator
from requests_html import HTML
from requests_html import HTMLSession

In [52]:
def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    except requests.exceptions.RequestException as e:
        print(e)

def get_results(query):
    query = urllib.parse.quote_plus(query)
    response = get_source("https://suggestqueries.google.com/complete/search?output=chrome&hl=en&q=" + query)
    results = json.loads(response.text)
    return results

def format_results(results):
    suggestions = []
    for index, value in enumerate(results[1]):
        suggestion = {'term': value, 'relevance': results[4]['google:suggestrelevance'][index]}
        suggestions.append(suggestion)
    return suggestions

def get_suggestions(query):
    results = get_results(query)
    results = format_results(results)
    results = sorted(results, key=lambda k: k['relevance'], reverse=True)
    return results

def google_autocomplete(query):
  results = get_suggestions(query)
  return results

In [53]:
%%time
search_term = "площ"
suggestions = google_autocomplete(search_term)

CPU times: user 135 ms, sys: 554 µs, total: 135 ms
Wall time: 344 ms


In [80]:
%%time
for suggestion in suggestions:
    print(suggestion['term'], suggestion['relevance'])

площадь треугольника 1250
площадь треугольника по трем сторонам 601
площадь треугольника через синус 600
площадь треугольника через высоту 561
площадь треугольника онлайн 560
площадь треугольника прямоугольного 559
площадь треугольника равностороннего 558
площадь треугольника по координатам 557
площадь треугольника через радиус вписанной окружности 556
площадь треугольной призмы 555
площадь треугольник 554
площадь треугольника через стороны 553
площадь треугольника вписанного в окружность 552
площадь треугольника через косинус 551
площадь треугольника все формулы 550
CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 2.02 ms


In [78]:
%%time
search_term = "площадь треуг"
suggestions = google_autocomplete(search_term)
for suggestion in suggestions:
    print(suggestion['term'], suggestion['relevance'])

площадь треугольника 1250
площадь треугольника по трем сторонам 601
площадь треугольника через синус 600
площадь треугольника через высоту 561
площадь треугольника онлайн 560
площадь треугольника прямоугольного 559
площадь треугольника равностороннего 558
площадь треугольника по координатам 557
площадь треугольника через радиус вписанной окружности 556
площадь треугольной призмы 555
площадь треугольник 554
площадь треугольника через стороны 553
площадь треугольника вписанного в окружность 552
площадь треугольника через косинус 551
площадь треугольника все формулы 550
CPU times: user 151 ms, sys: 740 µs, total: 152 ms
Wall time: 256 ms
